<a href="https://colab.research.google.com/github/AimanLakhani/NBA_Injury_Analysis/blob/main/NBA_Injury_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import altair as alt

In [ ]:
import re
def convert_google_sheet_url(url):
    # Regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    # If gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'

    # Replace using regex
    new_url = re.sub(pattern, replacement, url)

    return new_url

Import Injuries and Advanced Stats Data Sets

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/19W4VTTiCsErRY9w-y8JmKm8_rVDjafM2IghuDXrHUQc/edit#gid=1841468663')
injuries_df = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1eqt2CzaC6SQPfJSed6weqSuhG9aHbBoh/edit#gid=1390304836')
df_2011 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1mcynL7pSxacJtauwKu1ISuPmcshOA1sz/edit?rtpof=true')
df_2012 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1UTrh5T1ecjUV_81EWC999LWN0CoGrISj/edit?rtpof=true')
df_2013 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1FMktOY9fWjFbHgHtcJH7ASiYjTX_8bsq/edit?rtpof=true')
df_2014 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1mTHk6fKgv6-OoDUdmLZafWCubGXT60DO/edit?rtpof=true')
df_2015 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1nrIAFsQnU7p0mtN8t6ayaJcuIu2A2tp8/edit?rtpof=true')
df_2016 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/10XM-JsbXai9WJSSs3ICG9FVltjP72Fzj/edit?rtpof=true')
df_2017 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1p48371rjMlXUWZsEp1VyC3hz4nQU7NqI/edit?rtpof=true')
df_2018 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1BdFV9xvTLnemb9j8wbaDlhV6eRxd5g9i/edit?rtpof=true')
df_2019 = pd.read_csv(newurl)

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1epUYdLtrbeNN-Ey5SWL2lamkTBplirCc/edit#gid=249582725')
df_2020 = pd.read_csv(newurl)

Add players and the season stats

In [ ]:
newurl = convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1uDxDM-2vHkT-_v-P9uqcfMDARF8ROlJfYmj4q3-kU18/edit#gid=1641914757')
players = pd.read_csv(newurl)

### Start Analysis

In [ ]:
df_2011.columns


Index(['Rk', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'OeFG%', 'OTOV%', 'ORB%',
       'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'Arena', 'Attend.',
       'Attend./G'],
      dtype='object')

In [ ]:
# combine all of the advanced stats for each team from 2011 season to 2020
df_list = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020]
league_avg = pd.DataFrame(columns = ['Rk', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'OeFG%', 'OTOV%', 'ORB%',
       'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'Arena', 'Attend.',
       'Attend./G', 'year'])
for year in range(2011, 2021):
  df_list[year-2011]['year'] = year
  # add the league avgs to their own data frame
  league_avg.loc[len(league_avg.index)] = df_list[year-2011].iloc[-1]
  df_list[year-2011] = df_list[year-2011][['Rk', 'Team', 'Age', 'W', 'L', 'MOV', 'SOS', 'NRtg', 'Pace', '3PAr', 'year']]
  df_list[year-2011].insert(loc = 2, column = 'Win Percentage', value = df_list[year-2011]['W'] / (df_list[year-2011]['W'] + df_list[year-2011]['L']))
  df_list[year-2011].drop(['W', 'L'], axis=1, inplace=True)
  df_list[year-2011].drop(df_list[year-2011].index[-1], inplace = True)

<ipython-input-18-98a6929b7860>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[year-2011].drop(['W', 'L'], axis=1, inplace=True)
<ipython-input-18-98a6929b7860>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[year-2011].drop(df_list[year-2011].index[-1], inplace = True)
<ipython-input-18-98a6929b7860>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[year-2011].drop(['W', 'L'], axis=1, inplace=True)
<ipytho

In [ ]:
# combine advanced stats for all teams for all years
df_all = pd.concat(df_list)
df_all

,Rk,Team,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,year
0,1.0,Miami Heat*,0.707317,29.0,7.46,-0.71,8.2,90.9,0.235,2011
1,2.0,Chicago Bulls*,0.756098,26.4,7.32,-0.79,8.0,90.4,0.215,2011
2,3.0,Los Angeles Lakers*,0.695122,30.2,6.11,-0.10,6.7,90.7,0.220,2011
3,4.0,San Antonio Spurs*,0.743902,28.8,5.71,0.16,6.2,92.3,0.261,2011
4,5.0,Boston Celtics*,0.682927,29.5,5.37,-0.54,5.9,90.4,0.180,2011
...,...,...,...,...,...,...,...,...,...,...
25,26.0,New York Knicks,0.318182,24.5,-6.45,-0.26,-6.5,98.6,0.318,2020
26,27.0,Charlotte Hornets,0.353846,24.3,-6.75,-0.28,-7.0,95.8,0.399,2020
27,28.0,Atlanta Hawks,0.298507,24.1,-7.97,0.27,-7.6,103.0,0.398,2020
28,29.0,Cleveland Cavaliers,0.292308,25.0,-7.89,0.12,-7.9,98.7,0.362,2020


In [ ]:
# indicate that teams with '*' made playoffs
df_all['playoffs'] = df_all.apply(lambda x: 'yes' if str(x['Team'])[-1] == '*' else 'no', axis = 1)
df_all['Team'] = df_all['Team'].astype(str).apply(lambda x: str(x)[:-1] if str(x)[-1] == '*' else str(x))

In [ ]:
# sort by team and year
df_all.sort_values(['Team', 'year'], inplace = True)
df_all.reset_index(drop = True, inplace = True)

In [ ]:
df_all

,Rk,Team,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,year,playoffs
0,18.0,Atlanta Hawks,0.536585,27.4,-0.82,-0.28,-0.9,89.3,0.222,2011,yes
1,6.0,Atlanta Hawks,0.606061,27.9,3.44,-0.77,3.7,90.2,0.249,2012,yes
2,13.0,Atlanta Hawks,0.536585,27.1,0.40,-0.48,0.4,92.6,0.286,2013,yes
3,17.0,Atlanta Hawks,0.463415,27.5,-0.48,-0.40,-0.5,94.6,0.316,2014,yes
4,4.0,Atlanta Hawks,0.731707,27.8,5.43,-0.68,5.8,93.9,0.321,2015,yes
...,...,...,...,...,...,...,...,...,...,...,...
295,17.0,Washington Wizards,0.500000,27.3,-0.50,0.00,-0.5,98.5,0.282,2016,no
296,9.0,Washington Wizards,0.597561,26.0,1.80,-0.45,1.9,97.4,0.284,2017,yes
297,15.0,Washington Wizards,0.524390,26.9,0.59,-0.06,0.6,96.6,0.310,2018,yes
298,25.0,Washington Wizards,0.390244,26.5,-2.90,-0.40,-2.8,101.4,0.370,2019,no


In [ ]:
df_all = df_all[pd.notna(df_all['Rk'])]
df_all['Rk'] = pd.to_numeric(df_all['Rk'])
df_all

,Rk,Team,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,year,playoffs
0,18.0,Atlanta Hawks,0.536585,27.4,-0.82,-0.28,-0.9,89.3,0.222,2011,yes
1,6.0,Atlanta Hawks,0.606061,27.9,3.44,-0.77,3.7,90.2,0.249,2012,yes
2,13.0,Atlanta Hawks,0.536585,27.1,0.40,-0.48,0.4,92.6,0.286,2013,yes
3,17.0,Atlanta Hawks,0.463415,27.5,-0.48,-0.40,-0.5,94.6,0.316,2014,yes
4,4.0,Atlanta Hawks,0.731707,27.8,5.43,-0.68,5.8,93.9,0.321,2015,yes
...,...,...,...,...,...,...,...,...,...,...,...
295,17.0,Washington Wizards,0.500000,27.3,-0.50,0.00,-0.5,98.5,0.282,2016,no
296,9.0,Washington Wizards,0.597561,26.0,1.80,-0.45,1.9,97.4,0.284,2017,yes
297,15.0,Washington Wizards,0.524390,26.9,0.59,-0.06,0.6,96.6,0.310,2018,yes
298,25.0,Washington Wizards,0.390244,26.5,-2.90,-0.40,-2.8,101.4,0.370,2019,no


In [ ]:
injuries_df

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)
...,...,...,...,...,...
27100,2020-09-30,Lakers,Dion Waiters,NaN,activated from IL
27101,2020-10-02,Heat,NaN,Bam Adebayo,strained neck (DTD)
27102,2020-10-02,Heat,NaN,Goran Dragic,placed on IL with torn plantar fascia in left ...
27103,2020-10-02,Heat,Chris Silva,NaN,activated from IL


In [ ]:
injuries_df['Date'] = pd.to_datetime(injuries_df['Date'])
def nba_season_year(date):
    if date.month >= 7:  # NBA season starts after July
        return date.year + 1
    else:
        return date.year

# Apply the function to create the 'NBA Season Year' column
injuries_df['NBA Season Year'] = injuries_df['Date'].apply(nba_season_year)
injuries_df['NBA Season Year'] = injuries_df['NBA Season Year'].replace(2021, 2020) # Covid extended nba season into October
injuries_df

,Date,Team,Acquired,Relinquished,Notes,NBA Season Year
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...,2011
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely),2011
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...,2011
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely),2011
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely),2011
...,...,...,...,...,...,...
27100,2020-09-30,Lakers,Dion Waiters,NaN,activated from IL,2020
27101,2020-10-02,Heat,NaN,Bam Adebayo,strained neck (DTD),2020
27102,2020-10-02,Heat,NaN,Goran Dragic,placed on IL with torn plantar fascia in left ...,2020
27103,2020-10-02,Heat,Chris Silva,NaN,activated from IL,2020


In [ ]:
# Find the date when New Orleans Hornets changed name to Pelicans
pelicans_first_date = injuries_df[injuries_df.Team == 'Pelicans'].Date.values[0]

# Replace team names with dates before from 'Hornets' with 'Pelicans'
injuries_df.loc[(injuries_df['Date'] <= pelicans_first_date) & (injuries_df['Team'] == 'Hornets'), 'Team'] = 'Pelicans'

In [ ]:
# check to see if Hornets were replaced
injuries_df[injuries_df.Team == 'Pelicans']

,Date,Team,Acquired,Relinquished,Notes,NBA Season Year
54,2010-10-27,Pelicans,NaN,Aaron Gray,placed on IL,2011
55,2010-10-27,Pelicans,NaN,Joe Alexander,placed on IL,2011
56,2010-10-27,Pelicans,NaN,Quincy Pondexter,placed on IL,2011
157,2010-11-03,Pelicans,NaN,Peja Stojakovic,placed on IL,2011
158,2010-11-03,Pelicans,Quincy Pondexter,NaN,activated from IL,2011
...,...,...,...,...,...,...
26884,2020-08-07,Pelicans,NaN,Zion Williamson,placed on IL,2020
26913,2020-08-09,Pelicans,Zion Williamson,NaN,activated from IL,2020
26930,2020-08-10,Pelicans,NaN,Brandon Ingram,sore right knee (out for season),2020
26931,2020-08-10,Pelicans,NaN,Jrue Holiday,placed on IL with right elbow injury (out for ...,2020


In [ ]:
# now of the remaining entries we have two Charlotte teams: Hornets and Bobcats - change their name to Hornets for consistency
injuries_df.replace({'Bobcats': 'Hornets'}, inplace = True)

In [ ]:
# Make sure there are no Bobcats left
injuries_df[injuries_df.Team == 'Bobcats'].head()

,Date,Team,Acquired,Relinquished,Notes,NBA Season Year


In [ ]:
# keep only those on the injured list
injured = injuries_df.dropna(subset = 'Relinquished')
injured = injured.loc[:, injured.columns != 'Acquired']
injured

,Date,Team,Relinquished,Notes,NBA Season Year
0,2010-10-03,Bulls,Carlos Boozer,fractured bone in right pinky finger (out inde...,2011
1,2010-10-06,Pistons,Jonas Jerebko,torn right Achilles tendon (out indefinitely),2011
2,2010-10-06,Pistons,Terrico White,broken fifth metatarsal in right foot (out ind...,2011
3,2010-10-08,Blazers,Jeff Ayres,torn ACL in right knee (out indefinitely),2011
4,2010-10-08,Nets,Troy Murphy,strained lower back (out indefinitely),2011
...,...,...,...,...,...
27097,2020-09-22,Celtics,Romeo Langford,surgery on right wrist (out for season),2020
27098,2020-09-23,Heat,Gabe Vincent,sore right knee (DTD),2020
27099,2020-09-30,Heat,Bam Adebayo,strained left shoulder (DTD),2020
27101,2020-10-02,Heat,Bam Adebayo,strained neck (DTD),2020


In [ ]:
# keep those on the acquired list
healed = injuries_df.dropna(subset = 'Acquired')
healed = healed.loc[:, healed.columns != 'Relinquished']

healed

,Date,Team,Acquired,Notes,NBA Season Year
53,2010-10-27,Heat,Jerry Stackhouse,activated from IL,2011
81,2010-10-27,Rockets,Jermaine Taylor,activated from IL,2011
101,2010-10-29,Cavaliers,Samardo Samuels,activated from IL,2011
103,2010-10-29,Celtics,Luke Harangody,activated from IL,2011
105,2010-10-29,Grizzlies,Marc Gasol,activated from IL,2011
...,...,...,...,...,...
27091,2020-09-08,Bucks,Giannis Antetokounmpo,returned to lineup,2020
27094,2020-09-12,Lakers,JaVale McGee,returned to lineup,2020
27100,2020-09-30,Lakers,Dion Waiters,activated from IL,2020
27103,2020-10-02,Heat,Chris Silva,activated from IL,2020


In [ ]:
# drop irrelevant columns from dataset
players.drop('Unnamed: 0', axis = 1, inplace = True)
players

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
...,...,...,...,...,...,...,...
3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany


In [ ]:
# Since we are looking at players in 2010, the players we want are presumably less than 50 years old, or born after 1960
players = players[players['born'] > 1960]
players

,Player,height,weight,collage,born,birth_city,birth_state
1397,Joe Barry,193.0,88.0,Georgia Institute of Technology,1969.0,Oakland,California
1516,Isiah Thomas*,185.0,81.0,Indiana University,1961.0,Chicago,Illinois
1538,Terry Cummings,206.0,99.0,DePaul University,1961.0,Chicago,Illinois
1539,Quintin Dailey,190.0,81.0,University of San Francisco,1961.0,Baltimore,Maryland
1547,John Greig,201.0,97.0,University of Oregon,1961.0,Sacramento,California
...,...,...,...,...,...,...,...
3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany


In [ ]:
# drop irrelevant stats for league avg
league_avg.drop(['Rk', 'Team', 'W', 'L', 'PW', 'PL', 'ORtg', 'DRtg', 'MOV', 'SOS', 'SRS', 'NRtg', 'Arena', 'Attend.', 'Attend./G'], axis = 1, inplace = True)
league_avg

,Age,Pace,FTr,3PAr,TS%,OeFG%,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,year
0,26.7,92.1,0.3,0.222,0.541,0.498,13.4,26.4,0.229,0.498,13.4,73.6,0.229,2011
1,26.7,91.3,0.276,0.226,0.527,0.487,13.8,27.0,0.208,0.487,13.8,73.0,0.208,2012
2,26.7,92.0,0.27,0.243,0.535,0.496,13.7,26.5,0.204,0.496,13.7,73.5,0.204,2013
3,26.6,93.9,0.284,0.259,0.541,0.501,13.6,25.5,0.215,0.501,13.6,74.5,0.215,2014
4,26.8,93.9,0.273,0.268,0.534,0.496,13.3,25.1,0.205,0.496,13.3,74.9,0.205,2015
5,26.8,95.8,0.276,0.285,0.541,0.502,13.2,23.8,0.209,0.502,13.2,76.2,0.209,2016
6,26.6,96.4,0.271,0.316,0.552,0.514,12.7,23.3,0.209,0.514,12.7,76.7,0.209,2017
7,26.6,97.3,0.252,0.337,0.556,0.521,13.0,22.3,0.193,0.521,13.0,77.7,0.193,2018
8,26.4,100.0,0.259,0.359,0.56,0.524,12.4,22.9,0.198,0.524,12.4,77.1,0.198,2019
9,26.2,100.3,0.26,0.384,0.565,0.529,12.8,22.5,0.201,0.529,12.8,77.5,0.201,2020


##Next Steps:

Check the rate of injuries of nba players per season - done

*   Are they actually increasing

*   Find stats that are also changing and see if there's a correlation

Which teams are more injury-prone than the others?
*   easy case study - Tom Thibodeau Chicago Bulls Era (2010 - 2015)

How does style affect injuries?

*   check which relevant stats are linked to injuries

  * check for both teams over season and the league in general
*   Which teams are less injury prone and are there any stats that could help us correlate that to a style of play
*   Make this into an argument that a certain style of play or roster composition is better
  *   Link this with win %

Are star players missing more games?

*   We need to find a way to check this - there's a stat called 'Value Over Replacement Player' that we can use to filter down to just stars, but then we need to decide what that value is for 'stars.' What's the cutoff for being a star or not? We could also just do all-stars

*   Are the missing games due to injury or due to load management

Can a team still be successful after injuries?

*   Pick teams that made the playoffs for each individual year.

*   make a scatter plot for each year to see how well they did in the playoffs

  *   correlate how much they were injured (value over replacement player that was lost) and then with how well they did overall

* See which teams weren't competitive after injuries and find out why   









In [ ]:
# VISUAL TWO
import altair as alt

# Filter the data for the years from 2010 to 2020
injuries_filtered = injured[(injured['NBA Season Year'] >= 2010) & (injured['NBA Season Year'] <= 2020)]

# Group by Year and Team to get the count of injuries for each year and team
injuries_count = injuries_filtered.groupby(['NBA Season Year', 'Team']).size().reset_index(name='Injury Count')

# Define the selection
year_selection = alt.selection_single(
    name='Slide based on', fields=['NBA Season Year'], init={'NBA Season Year': 2011},
    bind=alt.binding_range(min=2011, max=2020, step=1)
)

# Define the base chart
base_chart = alt.Chart(injuries_count).transform_filter(
    year_selection
).mark_bar().encode(
    x='Injury Count:Q',
    y=alt.Y('Team:N', sort='-x'),
    color='Team:N',
    tooltip=['Team:N', 'Injury Count:Q']
).properties(
    width=600,
    height=400
)

text_chart = base_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='Injury Count:Q'

)

# Combine the base chart and text chart
combined_chart_2 = (base_chart + text_chart).add_selection(year_selection)



# Display the chart
combined_chart_2=combined_chart_2.properties(
    title='Number of Injuries by Team'
).configure_title(fontSize=20)
combined_chart_2

alt.LayerChart(...)

In [ ]:
# change team name for respective teams
df_all['Team'] = df_all['Team'].replace('New Orleans Hornets', 'New Orleans Pelicans')
df_all['Team'] = df_all['Team'].replace('Charlotte Bobcats', 'Charlotte Hornets')
df_all['Team'] = df_all['Team'].replace('New Jersey Nets', 'Brooklyn Nets')

# add team name column for joining
df_all['Team Name'] = df_all['Team'].str.split().str[-1]
df_all

,Rk,Team,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,year,playoffs,Team Name
0,18.0,Atlanta Hawks,0.536585,27.4,-0.82,-0.28,-0.9,89.3,0.222,2011,yes,Hawks
1,6.0,Atlanta Hawks,0.606061,27.9,3.44,-0.77,3.7,90.2,0.249,2012,yes,Hawks
2,13.0,Atlanta Hawks,0.536585,27.1,0.40,-0.48,0.4,92.6,0.286,2013,yes,Hawks
3,17.0,Atlanta Hawks,0.463415,27.5,-0.48,-0.40,-0.5,94.6,0.316,2014,yes,Hawks
4,4.0,Atlanta Hawks,0.731707,27.8,5.43,-0.68,5.8,93.9,0.321,2015,yes,Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...
295,17.0,Washington Wizards,0.500000,27.3,-0.50,0.00,-0.5,98.5,0.282,2016,no,Wizards
296,9.0,Washington Wizards,0.597561,26.0,1.80,-0.45,1.9,97.4,0.284,2017,yes,Wizards
297,15.0,Washington Wizards,0.524390,26.9,0.59,-0.06,0.6,96.6,0.310,2018,yes,Wizards
298,25.0,Washington Wizards,0.390244,26.5,-2.90,-0.40,-2.8,101.4,0.370,2019,no,Wizards


In [ ]:
# count the injuries per team per year
injuries_grouped = injured.groupby(['NBA Season Year', 'Team']).agg({'Relinquished': lambda x: list(x)}).reset_index()
injuries_grouped['Injury Count'] = injuries_grouped['Relinquished'].apply(lambda x: len(x))
injuries_grouped.rename(columns={'Relinquished': 'List of Injuries'}, inplace=True)

# Add injuries to df_all
injuries_stats = injuries_grouped.merge(df_all, left_on = ['NBA Season Year', 'Team'], right_on = ['year', 'Team Name'])
injuries_stats = injuries_stats[['Team_y', 'Team Name', 'year', 'Injury Count', 'List of Injuries', 'Rk', 'Win Percentage', 'Age', 'MOV', 'SOS', 'NRtg', 'Pace', '3PAr', 'playoffs']]
injuries_stats.rename(columns={'Team_y': 'Team'}, inplace=True)
injuries_stats

,Team,Team Name,year,Injury Count,List of Injuries,Rk,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,playoffs
0,Philadelphia 76ers,76ers,2011,33,"[Craig Brackins, Jodie Meeks, Andre Iguodala, ...",13.0,0.500000,24.7,1.50,-0.50,1.6,91.7,0.184,yes
1,Portland Trail Blazers,Blazers,2011,48,"[Jeff Ayres, Elliot Williams, Greg Oden, Joel ...",12.0,0.585366,26.7,1.52,0.32,1.7,87.9,0.228,yes
2,Milwaukee Bucks,Bucks,2011,98,"[Darington Hobson, Chris Douglas-Roberts, Jon ...",16.0,0.426829,26.7,-0.84,-0.17,-0.9,89.8,0.216,no
3,Chicago Bulls,Bulls,2011,33,"[Carlos Boozer, Carlos Boozer, Taj Gibson, Der...",2.0,0.756098,26.4,7.32,-0.79,8.0,90.4,0.215,yes
4,Cleveland Cavaliers,Cavaliers,2011,83,"[Christian Eyenga, Mo Williams, Samardo Samuel...",30.0,0.231707,26.6,-9.01,0.14,-9.6,93.0,0.224,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Phoenix Suns,Suns,2020,50,"[Cameron Johnson, Ty Jerome, Jalen Lecque, Ty ...",14.0,0.465753,24.6,0.22,0.35,0.3,101.3,0.361,no
296,Oklahoma City Thunder,Thunder,2020,50,"[Andre Roberson, Andre Roberson, Steven Adams,...",13.0,0.611111,25.5,1.99,0.34,2.0,98.8,0.353,yes
297,Minnesota Timberwolves,Timberwolves,2020,35,"[Keita Bates-Diop, Naz Reid, Jeff Teague, Jord...",24.0,0.296875,24.8,-4.30,0.28,-4.1,103.4,0.433,no
298,Golden State Warriors,Warriors,2020,56,"[Klay Thompson, Alec Burks, Alen Smailagic, Wi...",30.0,0.230769,24.4,-8.71,0.59,-8.6,100.3,0.355,no


In [ ]:
import numpy as np
import plotly.express as px

# Filter out entries with a team name of "undefined"
injuries_stats_filtered = injuries_stats[injuries_stats['Team Name'] != 'undefined']

# Filter out entries containing "..2 entries" in the Team Name column
injuries_stats_filtered = injuries_stats_filtered[~injuries_stats_filtered['Team Name'].str.contains('\.\.2 entries')]

# Compute the coefficients for the line of best fit
p = np.polyfit(injuries_stats_filtered['Pace'], injuries_stats_filtered['Injury Count'], 1)

# Create the scatter plot
fig = px.scatter(injuries_stats_filtered, x='Pace', y='Injury Count', color='Team Name', hover_data=['Team Name', 'year'])

# Add the line of best fit
x_values = np.array([injuries_stats_filtered['Pace'].min(), injuries_stats_filtered['Pace'].max()])
y_values = np.polyval(p, x_values)
fig.add_scatter(x=x_values, y=y_values, mode='lines', name='Overall Line of Best Fit', line=dict(color='black'))

# Update layout
fig.update_layout(
    title='Injury Count vs. Pace with Overall Line of Best Fit (Scatter Points Colored by Team)',
    width=800,
    height=600
)

# Show plot
fig.show()


In [ ]:
import plotly.graph_objects as go

# Assuming you have a DataFrame named `injuries_stats` containing the data

# Define colors for playoff and non-playoff teams
playoff_color = 'blue'
non_playoff_color = 'red'

# Create a list to store colors for each data point based on playoffs
colors = []
for playoff_status in injuries_stats['playoffs']:
    if playoff_status == 'yes':
        colors.append(playoff_color)
    else:
        colors.append(non_playoff_color)

# Create two separate traces for playoff and non-playoff teams
playoff_trace = go.Scatter3d(
    x=injuries_stats['year'][injuries_stats['playoffs'] == 'yes'],
    y=injuries_stats['Pace'][injuries_stats['playoffs'] == 'yes'],
    z=injuries_stats['Injury Count'][injuries_stats['playoffs'] == 'yes'],
    mode='markers',
    marker=dict(
        size=5,
        color=playoff_color
    ),
    name='Playoffs',
    hovertemplate='<b>Team:</b> %{text[0]}<br><b>Year:</b> %{text[1]}<br><b>Pace:</b> %{text[2]}<br><b>Injury Count:</b> %{text[3]}<extra></extra>',
    text=injuries_stats[injuries_stats['playoffs'] == 'yes'][['Team', 'year', 'Pace', 'Injury Count']].apply(lambda row: list(row), axis=1)
)

non_playoff_trace = go.Scatter3d(
    x=injuries_stats['year'][injuries_stats['playoffs'] == 'no'],
    y=injuries_stats['Pace'][injuries_stats['playoffs'] == 'no'],
    z=injuries_stats['Injury Count'][injuries_stats['playoffs'] == 'no'],
    mode='markers',
    marker=dict(
        size=5,
        color=non_playoff_color
    ),
    name='Non-Playoffs',
    hovertemplate='<b>Team:</b> %{text[0]}<br><b>Year:</b> %{text[1]}<br><b>Pace:</b> %{text[2]}<br><b>Injury Count:</b> %{text[3]}<extra></extra>',
    text=injuries_stats[injuries_stats['playoffs'] == 'no'][['Team', 'year', 'Pace', 'Injury Count']].apply(lambda row: list(row), axis=1)
)

# Create 3D scatter plot with both traces
fig_3 = go.Figure(data=[playoff_trace, non_playoff_trace])

# Update plot layout with legend
fig_3.update_layout(
    title="3-D Distribution of Year, Pace, and Injury Count",
    scene=dict(
        xaxis_title='Year',
        yaxis_title='Pace',
        zaxis_title='Injury Count'
    ),
    margin=dict(l=10, r=10, b=0, t=40),
    legend=dict(
        title='Playoff Status',
        yanchor='top',
        y=0.99,
        xanchor='left',
        x=0.01
    )
)

# Show the plot
fig_3.show()


In [ ]:
# prompt: convert every column in league_avg to float
league_avg = league_avg.astype('float')
league_avg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Age      10 non-null     float64
 1   Pace     10 non-null     float64
 2   FTr      10 non-null     float64
 3   3PAr     10 non-null     float64
 4   TS%      10 non-null     float64
 5   OeFG%    10 non-null     float64
 6   OTOV%    10 non-null     float64
 7   ORB%     10 non-null     float64
 8   OFT/FGA  10 non-null     float64
 9   DeFG%    10 non-null     float64
 10  DTOV%    10 non-null     float64
 11  DRB%     10 non-null     float64
 12  DFT/FGA  10 non-null     float64
 13  year     10 non-null     float64
dtypes: float64(14)
memory usage: 1.5 KB


In [ ]:
# count of injuries per year along with the season averages
injured_year_stats = injured.groupby('NBA Season Year').count()[['Team']].rename(columns = {'Team' : 'Injuries Count'}).merge(league_avg, left_on = 'NBA Season Year', right_on = 'year')
# how are injuries related to each stat
injured_year_stats.corr()[['Injuries Count']]

,Injuries Count
Injuries Count,1.000000
Age,0.324897
Pace,-0.535580
FTr,0.372446
3PAr,-0.518109
TS%,-0.399070
OeFG%,-0.420195
OTOV%,0.581772
ORB%,0.437344
OFT/FGA,0.338714


In [ ]:
injured_year_stats

,Injuries Count,Age,Pace,FTr,3PAr,TS%,OeFG%,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,year
0,1774,26.7,92.1,0.300,0.222,0.541,0.498,13.4,26.4,0.229,0.498,13.4,73.6,0.229,2011.0
1,1924,26.7,91.3,0.276,0.226,0.527,0.487,13.8,27.0,0.208,0.487,13.8,73.0,0.208,2012.0
2,1868,26.7,92.0,0.270,0.243,0.535,0.496,13.7,26.5,0.204,0.496,13.7,73.5,0.204,2013.0
3,2402,26.6,93.9,0.284,0.259,0.541,0.501,13.6,25.5,0.215,0.501,13.6,74.5,0.215,2014.0
4,1448,26.8,93.9,0.273,0.268,0.534,0.496,13.3,25.1,0.205,0.496,13.3,74.9,0.205,2015.0
5,1633,26.8,95.8,0.276,0.285,0.541,0.502,13.2,23.8,0.209,0.502,13.2,76.2,0.209,2016.0
6,1824,26.6,96.4,0.271,0.316,0.552,0.514,12.7,23.3,0.209,0.514,12.7,76.7,0.209,2017.0
7,1890,26.6,97.3,0.252,0.337,0.556,0.521,13.0,22.3,0.193,0.521,13.0,77.7,0.193,2018.0
8,1437,26.4,100.0,0.259,0.359,0.560,0.524,12.4,22.9,0.198,0.524,12.4,77.1,0.198,2019.0
9,1358,26.2,100.3,0.260,0.384,0.565,0.529,12.8,22.5,0.201,0.529,12.8,77.5,0.201,2020.0


In [ ]:
#merge df_all and injuries_grouped
injuries_per_team = injuries_grouped.merge(df_all, left_on = ['NBA Season Year', 'Team'], right_on = ['year', 'Team Name'])[[ 'Team_y', 'year', 'Injury Count', 'Rk',
                                                              'Win Percentage', 'Age', 'MOV', 'SOS', 'NRtg', 'Pace', '3PAr', 'playoffs']].rename(columns = {'Team_y': 'Team'})
injuries_per_team.sort_values('Injury Count', ascending=False)

,Team,year,Injury Count,Rk,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,playoffs
104,Los Angeles Lakers,2014,196,27.0,0.329268,26.8,-6.29,0.96,-6.4,98.7,0.291,no
92,Milwaukee Bucks,2014,160,29.0,0.182927,24.7,-8.18,-0.23,-8.8,91.8,0.231,no
113,Houston Rockets,2014,154,7.0,0.658537,25.4,4.56,0.50,4.7,96.3,0.330,yes
87,Minnesota Timberwolves,2013,136,20.0,0.378049,26.4,-2.37,0.54,-2.5,92.8,0.220,no
50,New Orleans Pelicans,2012,134,24.0,0.318182,25.5,-3.76,0.65,-4.2,88.3,0.152,no
...,...,...,...,...,...,...,...,...,...,...,...,...
294,San Antonio Spurs,2020,24,17.0,0.450704,27.6,-1.11,0.46,-1.1,100.5,0.318,no
141,Detroit Pistons,2015,23,20.0,0.390244,25.9,-1.00,-0.39,-1.1,92.8,0.290,no
258,Denver Nuggets,2019,23,8.0,0.658537,24.9,3.95,0.24,4.1,97.7,0.348,yes
280,Charlotte Hornets,2020,22,27.0,0.353846,24.3,-6.75,-0.28,-7.0,95.8,0.399,no


In [ ]:
injuries_per_team.corr(numeric_only=True)[['Injury Count']]

,Injury Count
year,-0.213996
Injury Count,1.000000
Rk,0.064617
Win Percentage,-0.089284
Age,0.071458
MOV,-0.065705
SOS,-0.037537
NRtg,-0.071719
Pace,-0.149639
3PAr,-0.085582


In [ ]:
# VISUAL ONE
import altair as alt
import pandas as pd

# Assuming your DataFrame is named injuries_per_team
# Transpose the DataFrame
transposed_df = injuries_per_team.pivot(index='year', columns='Team', values='Injury Count').reset_index()

# Melt the DataFrame to Altair-friendly format
alt_data = pd.melt(transposed_df, id_vars=['year'], var_name='Team', value_name='Injury Count')

# Calculate average injury count per year
average_df = alt_data.groupby('year')['Injury Count'].mean().reset_index()

# Create the scatterplot
scatterplot = alt.Chart(alt_data).mark_circle().encode(
    x='year:O',
    y='Injury Count:Q',
    color='Team:N',
    tooltip=['Team', 'year', 'Injury Count']
).properties(
    width=600,
    height=400,
    title='Injuries per Team Over Years'
)

# Add average point
average_point = alt.Chart(average_df).mark_point(color='red', size=100).encode(
    x='year:O',
    y='Injury Count:Q'
)

# Add average line
average_line = alt.Chart(average_df).mark_line(color='red').encode(
    x='year:O',
    y='Injury Count:Q'
)
spike_annotation = alt.Chart(pd.DataFrame({'year': [2014], 'Injury Count': [300]})).mark_text(
    align='left', baseline='middle', dx=-60, dy=80, fontSize=20, color='red'
).encode(
    x='year:O',
    y='Injury Count:Q',
    text=alt.value('Spike in 2014')
)

# Combine scatterplot, average point, and average line
chart = (scatterplot + average_point + average_line + spike_annotation).properties(
    width=600,
    height=400,
    title='Injuries per Team Over Years with Average'
)

# Show the plot
chart

alt.LayerChart(...)

Extract data from internet

In [ ]:
# needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
def url_to_df(url, year):
  # collect HTML data
  html = urlopen(url)

  # create beautiful soup object from HTML
  soup = BeautifulSoup(html, features="lxml")
  # use getText()to extract the headers into a list
  headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]
  # remove irrelevant headers
  headers = [header for header in headers if not header == '\xa0']
  headers = headers[1:]
  # get rows from table
  rows = soup.findAll('tr')[1:]
  rows_data = [[td.getText() for td in rows[i].findAll('td')]
                      for i in range(len(rows))]

  rows_data = [[strng for strng in sublist if strng] for sublist in rows_data]
  df = pd.DataFrame(rows_data, columns = headers)
  df['year'] = year
  return df


In [ ]:
 players_2011 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2011_advanced.html#advanced_stats', 2011)

In [ ]:
players_2012 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2012_advanced.html#advanced_stats', 2012)

In [ ]:
players_2013 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2013_advanced.html#advanced_stats', 2013)

In [ ]:
players_2014 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2014_advanced.html#advanced_stats', 2014)

In [ ]:
players_2015 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2015_advanced.html#advanced_stats', 2015)

In [ ]:
players_2016 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2016_advanced.html#advanced_stats', 2016)

In [ ]:
players_2017 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2017_advanced.html#advanced_stats', 2017)

In [ ]:
players_2018 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2018_advanced.html#advanced_stats', 2018)

In [ ]:
players_2019 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2019_advanced.html#advanced_stats', 2019)

In [ ]:
players_2020 = url_to_df('https://www.basketball-reference.com/leagues/NBA_2020_advanced.html#advanced_stats', 2020)

In [ ]:
# make a list of all players
df_list = [players_2011, players_2012, players_2013, players_2014, players_2015, players_2016, players_2017, players_2018, players_2019, players_2020]
all_players = pd.concat(df_list)

In [ ]:
all_players['Player'] = all_players['Player'].astype(str).apply(lambda x: str(x)[:-1] if str(x)[-1] == '*' else str(x))

In [ ]:
all_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6448 entries, 0 to 676
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  6448 non-null   object
 1   Pos     6207 non-null   object
 2   Age     6207 non-null   object
 3   Tm      6207 non-null   object
 4   G       6207 non-null   object
 5   MP      6207 non-null   object
 6   PER     6207 non-null   object
 7   TS%     6207 non-null   object
 8   3PAr    6207 non-null   object
 9   FTr     6207 non-null   object
 10  ORB%    6207 non-null   object
 11  DRB%    6207 non-null   object
 12  TRB%    6207 non-null   object
 13  AST%    6206 non-null   object
 14  STL%    6206 non-null   object
 15  BLK%    6206 non-null   object
 16  TOV%    6206 non-null   object
 17  USG%    6206 non-null   object
 18  OWS     6206 non-null   object
 19  DWS     6206 non-null   object
 20  WS      6206 non-null   object
 21  WS/48   6206 non-null   object
 22  OBPM    6178 non-null   object

In [ ]:
all_players = all_players.apply(pd.to_numeric, errors='ignore')

In [ ]:
# if a player got traded, they have multiple entires in a given year
all_players.groupby(['Player', 'year']).mean(numeric_only = True).reset_index()

,Player,year,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,A.J. Hammons,2017,24.0,22.0,163.000000,8.4,0.472,0.238,0.476000,5.400000,...,16.400000,17.6,-0.200000,0.2,0.000000,-0.001000,-6.600000,0.000000,-6.600000,-0.2
1,A.J. Price,2011,24.0,50.0,795.000000,10.7,0.454,0.466,0.253000,2.300000,...,13.000000,22.7,-0.400000,0.7,0.300000,0.020000,-1.900000,-1.100000,-2.900000,-0.2
2,A.J. Price,2012,25.0,44.0,568.000000,11.5,0.454,0.506,0.201000,2.600000,...,14.500000,17.7,0.200000,0.5,0.700000,0.063000,-0.600000,-0.500000,-1.000000,0.1
3,A.J. Price,2013,26.0,57.0,1278.000000,12.4,0.501,0.484,0.150000,1.700000,...,12.700000,18.0,1.000000,1.2,2.200000,0.084000,-0.200000,-0.400000,-0.600000,0.5
4,A.J. Price,2014,27.0,28.0,99.000000,9.7,0.469,0.478,0.043000,1.100000,...,13.000000,23.2,-0.100000,0.0,0.000000,-0.012000,-2.400000,-2.400000,-4.800000,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4937,Ömer Aşık,2014,27.0,48.0,968.000000,14.0,0.570,0.000,0.663000,12.300000,...,19.400000,13.8,1.000000,1.5,2.400000,0.121000,-1.200000,-0.300000,-1.500000,0.1
4938,Ömer Aşık,2015,28.0,76.0,1982.000000,15.5,0.545,0.000,0.546000,14.000000,...,15.700000,14.0,2.800000,2.2,5.000000,0.120000,-0.400000,-0.900000,-1.200000,0.4
4939,Ömer Aşık,2016,29.0,68.0,1178.000000,11.0,0.551,0.000,0.574000,10.900000,...,19.700000,11.4,0.700000,1.0,1.700000,0.070000,-2.800000,-1.000000,-3.800000,-0.5
4940,Ömer Aşık,2017,30.0,31.0,482.000000,9.8,0.517,0.000,0.600000,10.400000,...,14.600000,8.8,0.400000,0.6,1.000000,0.100000,-3.600000,-0.100000,-3.700000,-0.2


In [ ]:
all_players.groupby('year').mean(numeric_only = True)

,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
year,,,,,,,,,,,,,,,,,,,,,
2011,26.736000,47.009600,1098.793600,12.200160,0.508651,0.219171,0.303000,5.501760,14.230560,9.869760,...,14.099200,18.334754,1.095680,1.097920,2.194560,0.067459,-1.652412,-0.197910,-1.851768,0.520579
2012,26.633394,40.059891,920.441016,12.731579,0.501494,0.216261,0.281416,5.677677,14.530853,10.097641,...,14.644465,18.488929,0.981851,0.923775,1.907441,0.071813,-1.500726,-0.117060,-1.617060,0.444646
2013,26.535777,49.219895,1112.923211,12.106632,0.495824,0.353134,0.329290,5.791623,14.149564,9.898778,...,14.395455,18.689949,1.169909,1.096853,2.270280,0.044470,-1.796316,-0.355009,-2.153076,0.539543
2014,26.472995,46.780687,1066.883797,12.200655,0.500903,0.286393,0.301326,5.403110,14.520131,9.972504,...,13.940098,18.216173,1.060404,1.024386,2.126350,0.047971,-1.652475,-0.259934,-1.917053,0.513411
2015,26.500768,45.960061,1041.775730,12.549002,0.563866,0.408089,0.358439,5.063902,14.087097,9.553303,...,13.445315,18.979430,1.084155,1.015207,2.111982,0.046241,-1.490769,-0.254545,-1.747766,0.493529
2016,26.823529,49.162630,1104.809689,13.217647,0.567915,0.372057,0.349960,4.881315,14.675952,9.749827,...,13.252249,18.561503,1.160727,1.111419,2.270588,0.079938,-1.149217,-0.146783,-1.297217,0.525043
2017,26.411765,48.431933,1090.682353,12.731597,0.525059,0.488563,0.364380,4.908908,14.973613,9.935462,...,12.845714,18.500476,1.056471,1.127563,2.185042,0.073376,-1.446037,-0.151096,-1.599831,0.535919
2018,26.195783,43.281627,972.881024,12.717771,0.525533,0.358970,0.257364,4.811898,14.898795,9.821687,...,12.846084,18.572986,1.002861,0.970482,1.975151,0.071161,-1.251212,-0.189697,-1.440000,0.478030
2019,26.142655,42.882768,972.257062,12.745763,0.526973,0.498732,0.307079,4.979802,15.047034,10.012006,...,11.919633,18.490672,0.948729,0.943644,1.893927,0.071562,-1.447009,-0.336895,-1.786182,0.449573


In [ ]:
injured_per_year = injured.groupby('NBA Season Year').count()[['Team']].rename(columns = {'Team' : 'Injuries Count'})
injured_per_year

,Injuries Count
NBA Season Year,
2011,1774
2012,1924
2013,1868
2014,2402
2015,1448
2016,1633
2017,1824
2018,1890
2019,1437


In [ ]:
###player_salaries = player_salaries[(player_salaries['season'] >= 2011) & (player_salaries['season'] <= 2020)]
#player_salaries = player_salaries[player_salaries['rank'] <= 50]
#player_salaries.reset_index(inplace = True)
#player_salaries.drop(columns = ['level_0'])

In [ ]:
#player_salaries.drop(columns = ['index'], inplace = True)

In [ ]:
#player_salaries

In [ ]:
"""top50_year_stats = top50_players.groupby('year').mean(numeric_only = True)
injured_top50 = injured_per_year.merge(top50_year_stats, left_on = 'NBA Season Year', right_on = 'year')
injured_top50.corr()[['Injuries Count']]"""

"top50_year_stats = top50_players.groupby('year').mean(numeric_only = True)\ninjured_top50 = injured_per_year.merge(top50_year_stats, left_on = 'NBA Season Year', right_on = 'year')\ninjured_top50.corr()[['Injuries Count']]"

In [ ]:
injured['month'] = injured['Date'].dt.month
injured

,Date,Team,Relinquished,Notes,NBA Season Year,month
0,2010-10-03,Bulls,Carlos Boozer,fractured bone in right pinky finger (out inde...,2011,10
1,2010-10-06,Pistons,Jonas Jerebko,torn right Achilles tendon (out indefinitely),2011,10
2,2010-10-06,Pistons,Terrico White,broken fifth metatarsal in right foot (out ind...,2011,10
3,2010-10-08,Blazers,Jeff Ayres,torn ACL in right knee (out indefinitely),2011,10
4,2010-10-08,Nets,Troy Murphy,strained lower back (out indefinitely),2011,10
...,...,...,...,...,...,...
27097,2020-09-22,Celtics,Romeo Langford,surgery on right wrist (out for season),2020,9
27098,2020-09-23,Heat,Gabe Vincent,sore right knee (DTD),2020,9
27099,2020-09-30,Heat,Bam Adebayo,strained left shoulder (DTD),2020,9
27101,2020-10-02,Heat,Bam Adebayo,strained neck (DTD),2020,10


In [ ]:
injured_per_month = injured.groupby(['NBA Season Year', 'month','Team']).count().iloc[:, 0: 1].rename(columns = {'Date': 'Injuries_per_month'})

In [ ]:
injured_per_month

Injuries_per_month
NBA Season Year month Team                            
2011            1     76ers                          7
                      Blazers                        7
                      Bucks                         24
                      Bulls                          3
                      Cavaliers                     26
...                                                ...
2020            12    Suns                          12
                      Thunder                        8
                      Timberwolves                  11
                      Warriors                      13
                      Wizards                       14

[2232 rows x 1 columns]

In [ ]:
# VISUAL FOUR
import altair as alt

# Sample data (replace this with your actual data)
# injured_per_month = injured.groupby(['NBA Season Year', 'month']).count().iloc[:, 0: 1].rename(columns = {'Date': 'Injuries_per_month'})

# Define the custom order of months
custom_month_order = [10, 11, 12, 1, 2, 3, 4, 5]

# Create a selection for the clicked year
click = alt.selection_multi(encodings=['color'])

# Create the bar chart
bars = alt.Chart(injured_per_month.reset_index()).mark_bar().encode(
    x=alt.X('month:O', axis=alt.Axis(title='Month', labelAngle=0), scale=alt.Scale(domain=custom_month_order)),
    y='sum(Injuries_per_month):Q',
    color=alt.condition(click, 'NBA Season Year:O', alt.value('lightgray'), legend=None)  # Remove legend for year
).properties(
    width=600,
    height=400
).add_selection(
    click
)

bars.configure_axis(
    grid=False
).configure_view(
    strokeOpacity=0
)

# Create the separate graph for team distribution
team_distribution = alt.Chart(injured_per_month.reset_index()).mark_bar().encode(
    x=alt.X('sum(Injuries_per_month):Q', axis=alt.Axis(title='Total Injuries'), sort='ascending'),  # Sort from least to greatest
    y=alt.Y('Team:N', sort='-x'),  # Sort bars by total injuries
    tooltip=['Team:N', 'sum(Injuries_per_month):Q']
).transform_filter(
    click
)

team_distribution.properties(
    width=600,
    height=400
)

# Create legend for year
legend_year = alt.Chart(injured_per_month.reset_index()).mark_point().encode(
    y=alt.Y('NBA Season Year:O', axis=alt.Axis(title='Year')),
    color=alt.Color('NBA Season Year:O', legend=alt.Legend(title='Year Shaded By'))
).add_selection(
    click
)

# Combine the two charts
combined_charts = bars | team_distribution | legend_year


chart_annotation = alt.Chart().mark_text(
    x=300,  # Position in the middle of the combined chart
    y=0,
    text="Steady increase until February (Month 2) due to The All-Star break",
    align='center',
    fontSize=25,
    color='black'
)

combined_charts_with_annotation4 = (combined_charts & chart_annotation)

combined_charts_with_annotation4


alt.VConcatChart(...)

Playing with data

In [ ]:
injured_per_month

Injuries_per_month
NBA Season Year month Team                            
2011            1     76ers                          7
                      Blazers                        7
                      Bucks                         24
                      Bulls                          3
                      Cavaliers                     26
...                                                ...
2020            12    Suns                          12
                      Thunder                        8
                      Timberwolves                  11
                      Warriors                      13
                      Wizards                       14

[2232 rows x 1 columns]

In [ ]:
injuries_per_team.sort_values('Injury Count', ascending = False)

,Team,year,Injury Count,Rk,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,playoffs
104,Los Angeles Lakers,2014,196,27.0,0.329268,26.8,-6.29,0.96,-6.4,98.7,0.291,no
92,Milwaukee Bucks,2014,160,29.0,0.182927,24.7,-8.18,-0.23,-8.8,91.8,0.231,no
113,Houston Rockets,2014,154,7.0,0.658537,25.4,4.56,0.50,4.7,96.3,0.330,yes
87,Minnesota Timberwolves,2013,136,20.0,0.378049,26.4,-2.37,0.54,-2.5,92.8,0.220,no
50,New Orleans Pelicans,2012,134,24.0,0.318182,25.5,-3.76,0.65,-4.2,88.3,0.152,no
...,...,...,...,...,...,...,...,...,...,...,...,...
294,San Antonio Spurs,2020,24,17.0,0.450704,27.6,-1.11,0.46,-1.1,100.5,0.318,no
141,Detroit Pistons,2015,23,20.0,0.390244,25.9,-1.00,-0.39,-1.1,92.8,0.290,no
258,Denver Nuggets,2019,23,8.0,0.658537,24.9,3.95,0.24,4.1,97.7,0.348,yes
280,Charlotte Hornets,2020,22,27.0,0.353846,24.3,-6.75,-0.28,-7.0,95.8,0.399,no


In [ ]:
# combine all of the advanced stats for each team from 2011 season to 2020
df_list = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020]
league_avg = pd.DataFrame(columns = ['Rk', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'OeFG%', 'OTOV%', 'ORB%',
       'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'Arena', 'Attend.',
       'Attend./G', 'year'])
for year in range(2011, 2021):
  df_list[year-2011]['year'] = year
  df_list[year-2011] = df_list[year-2011][['Rk', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'OeFG%', 'OTOV%', 'ORB%',
       'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'year']]
  df_list[year-2011].insert(loc = 2, column = 'Win Percentage', value = df_list[year-2011]['W'] / (df_list[year-2011]['W'] + df_list[year-2011]['L']))
  df_list[year-2011].drop(['W', 'L'], axis=1, inplace=True)
  df_list[year-2011].drop(df_list[year-2011].index[-1], inplace = True)

<ipython-input-75-aa8b5cf6ea76>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-75-aa8b5cf6ea76>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-75-aa8b5cf6ea76>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-75-aa8b5cf6ea76>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [ ]:
df_ = pd.concat(df_list)

In [ ]:
df_

,Rk,Team,Win Percentage,Age,PW,PL,MOV,SOS,SRS,ORtg,...,TS%,OeFG%,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,year
0,1.0,Miami Heat*,0.707317,29.0,61,21,7.46,-0.71,6.76,111.7,...,0.573,0.524,13.5,25.2,0.279,0.475,12.6,75.5,0.223,2011
1,2.0,Chicago Bulls*,0.756098,26.4,61,21,7.32,-0.79,6.53,108.3,...,0.541,0.501,13.5,29.4,0.227,0.463,13.6,76.2,0.222,2011
2,3.0,Los Angeles Lakers*,0.695122,30.2,58,24,6.11,-0.10,6.01,111.0,...,0.545,0.502,12.3,29.2,0.228,0.477,12.8,72.3,0.181,2011
3,4.0,San Antonio Spurs*,0.743902,28.8,56,26,5.71,0.16,5.86,111.8,...,0.567,0.527,12.8,24.9,0.229,0.491,12.6,74.6,0.190,2011
4,5.0,Boston Celtics*,0.682927,29.5,57,25,5.37,-0.54,4.83,106.2,...,0.561,0.519,14.5,21.1,0.235,0.469,14.8,74.7,0.235,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26.0,New York Knicks,0.318182,24.5,20,46,-6.45,-0.26,-6.72,106.5,...,0.531,0.501,12.6,25.8,0.182,0.541,12.4,78.3,0.224,2020
26,27.0,Charlotte Hornets,0.353846,24.3,19,46,-6.75,-0.28,-7.03,106.3,...,0.539,0.504,13.3,23.9,0.188,0.546,13.1,74.4,0.159,2020
27,28.0,Atlanta Hawks,0.298507,24.1,18,49,-7.97,0.27,-7.71,107.2,...,0.554,0.515,13.8,21.6,0.204,0.543,12.7,74.9,0.233,2020
28,29.0,Cleveland Cavaliers,0.292308,25.0,18,47,-7.89,0.12,-7.77,107.5,...,0.553,0.522,14.6,24.6,0.172,0.560,11.7,77.4,0.164,2020


In [ ]:
df_['playoffs'] = df_.apply(lambda x: 'yes' if str(x['Team'])[-1] == '*' else 'no', axis = 1)
df_['Team'] = df_['Team'].astype(str).apply(lambda x: str(x)[:-1] if str(x)[-1] == '*' else str(x))

In [ ]:
df_['Team'] = df_['Team'].replace('New Orleans Hornets', 'New Orleans Pelicans')
df_['Team'] = df_['Team'].replace('Charlotte Bobcats', 'Charlotte Hornets')
df_['Team'] = df_['Team'].replace('New Jersey Nets', 'Brooklyn Nets')

In [ ]:
df_

,Rk,Team,Win Percentage,Age,PW,PL,MOV,SOS,SRS,ORtg,...,OeFG%,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,year,playoffs
0,1.0,Miami Heat,0.707317,29.0,61,21,7.46,-0.71,6.76,111.7,...,0.524,13.5,25.2,0.279,0.475,12.6,75.5,0.223,2011,yes
1,2.0,Chicago Bulls,0.756098,26.4,61,21,7.32,-0.79,6.53,108.3,...,0.501,13.5,29.4,0.227,0.463,13.6,76.2,0.222,2011,yes
2,3.0,Los Angeles Lakers,0.695122,30.2,58,24,6.11,-0.10,6.01,111.0,...,0.502,12.3,29.2,0.228,0.477,12.8,72.3,0.181,2011,yes
3,4.0,San Antonio Spurs,0.743902,28.8,56,26,5.71,0.16,5.86,111.8,...,0.527,12.8,24.9,0.229,0.491,12.6,74.6,0.190,2011,yes
4,5.0,Boston Celtics,0.682927,29.5,57,25,5.37,-0.54,4.83,106.2,...,0.519,14.5,21.1,0.235,0.469,14.8,74.7,0.235,2011,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26.0,New York Knicks,0.318182,24.5,20,46,-6.45,-0.26,-6.72,106.5,...,0.501,12.6,25.8,0.182,0.541,12.4,78.3,0.224,2020,no
26,27.0,Charlotte Hornets,0.353846,24.3,19,46,-6.75,-0.28,-7.03,106.3,...,0.504,13.3,23.9,0.188,0.546,13.1,74.4,0.159,2020,no
27,28.0,Atlanta Hawks,0.298507,24.1,18,49,-7.97,0.27,-7.71,107.2,...,0.515,13.8,21.6,0.204,0.543,12.7,74.9,0.233,2020,no
28,29.0,Cleveland Cavaliers,0.292308,25.0,18,47,-7.89,0.12,-7.77,107.5,...,0.522,14.6,24.6,0.172,0.560,11.7,77.4,0.164,2020,no


In [ ]:
#merge df_all and injuries_grouped
injuries_per_team2 = injuries_grouped.merge(df_all, left_on = ['NBA Season Year', 'Team'], right_on = ['year', 'Team Name']).rename(columns = {'Team_y': 'Team'})
injuries_per_team2.sort_values('Injury Count', ascending=False)

,NBA Season Year,Team_x,List of Injuries,Injury Count,Rk,Team,Win Percentage,Age,MOV,SOS,NRtg,Pace,3PAr,year,playoffs,Team Name
104,2014,Lakers,"[Elias Harris, Kobe Bryant, Ryan Kelly, Steve ...",196,27.0,Los Angeles Lakers,0.329268,26.8,-6.29,0.96,-6.4,98.7,0.291,2014,no,Lakers
92,2014,Bucks,"[Ekpe Udoh, Carlos Delfino, Carlos Delfino, Ek...",160,29.0,Milwaukee Bucks,0.182927,24.7,-8.18,-0.23,-8.8,91.8,0.231,2014,no,Bucks
113,2014,Rockets,"[Greg Smith, Isaiah Canaan, Robert Covington, ...",154,7.0,Houston Rockets,0.658537,25.4,4.56,0.50,4.7,96.3,0.330,2014,yes,Rockets
87,2013,Timberwolves,"[Kevin Love, Kevin Love, Ricky Rubio, Jose Bar...",136,20.0,Minnesota Timberwolves,0.378049,26.4,-2.37,0.54,-2.5,92.8,0.220,2013,no,Timberwolves
50,2012,Pelicans,"[DaJuan Summers, DaJuan Summers, Gustavo Ayon,...",134,24.0,New Orleans Pelicans,0.318182,25.5,-3.76,0.65,-4.2,88.3,0.152,2012,no,Pelicans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2020,Spurs,"[Derrick White, Derrick White, Chimezie Metu, ...",24,17.0,San Antonio Spurs,0.450704,27.6,-1.11,0.46,-1.1,100.5,0.318,2020,no,Spurs
141,2015,Pistons,"[Jodie Meeks, Cartier Martin, Jodie Meeks, Gig...",23,20.0,Detroit Pistons,0.390244,25.9,-1.00,-0.39,-1.1,92.8,0.290,2015,no,Pistons
258,2019,Nuggets,"[Isaiah Thomas, Jarred Vanderbilt, Isaiah Thom...",23,8.0,Denver Nuggets,0.658537,24.9,3.95,0.24,4.1,97.7,0.348,2019,yes,Nuggets
280,2020,Hornets,"[Nicolas Batum, Nicolas Batum, Michael Kidd-Gi...",22,27.0,Charlotte Hornets,0.353846,24.3,-6.75,-0.28,-7.0,95.8,0.399,2020,no,Hornets


In [ ]:
# prompt: for injuries_per_team2 convert playoffs to 0s if ''no" or 1 if 'win'

injuries_per_team2['playoffs'] = injuries_per_team2['playoffs'].apply(lambda x: 0 if x == 'no' else 1)

In [ ]:
injuries_per_team2.corr(numeric_only=True)['Injury Count']

NBA Season Year   -0.213996
Injury Count       1.000000
Rk                 0.064617
Win Percentage    -0.089284
Age                0.071458
MOV               -0.065705
SOS               -0.037537
NRtg              -0.071719
Pace              -0.149639
3PAr              -0.085582
year              -0.213996
playoffs          -0.106476
Name: Injury Count, dtype: float64

In [ ]:
injured_year_stats.corr()['Injuries Count']

Injuries Count    1.000000
Age               0.324897
Pace             -0.535580
FTr               0.372446
3PAr             -0.518109
TS%              -0.399070
OeFG%            -0.420195
OTOV%             0.581772
ORB%              0.437344
OFT/FGA           0.338714
DeFG%            -0.420195
DTOV%             0.581772
DRB%             -0.437344
DFT/FGA           0.338714
year             -0.512667
Name: Injuries Count, dtype: float64

In [ ]:
injuries_TOV = injured_year_stats[['Injuries Count', 'year', 'OTOV%']]

In [ ]:
injuries_TOV

,Injuries Count,year,OTOV%
0,1774,2011.0,13.4
1,1924,2012.0,13.8
2,1868,2013.0,13.7
3,2402,2014.0,13.6
4,1448,2015.0,13.3
5,1633,2016.0,13.2
6,1824,2017.0,12.7
7,1890,2018.0,13.0
8,1437,2019.0,12.4
9,1358,2020.0,12.8


In [ ]:
inj_merge = injuries_per_team[['Team', 'year', 'Injury Count']]
inj_merge

,Team,year,Injury Count
0,Philadelphia 76ers,2011,33
1,Portland Trail Blazers,2011,48
2,Milwaukee Bucks,2011,98
3,Chicago Bulls,2011,33
4,Cleveland Cavaliers,2011,83
...,...,...,...
295,Phoenix Suns,2020,50
296,Oklahoma City Thunder,2020,50
297,Minnesota Timberwolves,2020,35
298,Golden State Warriors,2020,56


In [ ]:
advanced_injuries = df_.merge(inj_merge, left_on = ['year', 'Team'], right_on = ['year', 'Team']).rename(columns = {'Team_y': 'Team'})
advanced_injuries.sort_values('Injury Count', ascending=False, inplace = True)
advanced_injuries.corr(numeric_only=True)['Injury Count']

Rk                0.064617
Win Percentage   -0.089284
Age               0.071458
PW               -0.062130
PL                0.089248
MOV              -0.065705
SOS              -0.037537
SRS              -0.070681
ORtg             -0.159827
DRtg             -0.070882
NRtg             -0.071719
Pace             -0.149639
FTr               0.097903
3PAr             -0.085582
TS%              -0.064408
OeFG%            -0.082055
OTOV%             0.239632
ORB%             -0.013979
OFT/FGA           0.091412
DeFG%            -0.108857
DTOV%             0.084072
DRB%             -0.188847
DFT/FGA           0.071551
year             -0.213996
Injury Count      1.000000
Name: Injury Count, dtype: float64

In [ ]:
advanced_injuries.columns

Index(['Rk', 'Team', 'Win Percentage', 'Age', 'PW', 'PL', 'MOV', 'SOS', 'SRS',
       'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'OeFG%', 'OTOV%',
       'ORB%', 'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'year',
       'playoffs', 'Injury Count'],
      dtype='object')

In [ ]:
advanced_injuries.sort_values(['playoffs', 'Injury Count'], ascending = False).head()

,Rk,Team,Win Percentage,Age,PW,PL,MOV,SOS,SRS,ORtg,...,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,year,playoffs,Injury Count
96,7.0,Houston Rockets,0.658537,25.4,53,29,4.56,0.50,5.06,111.0,...,14.6,27.4,0.275,0.489,12.5,74.1,0.193,2014,yes,154
181,2.0,San Antonio Spurs,0.743902,29.6,60,22,7.20,-0.06,7.13,111.1,...,12.6,24.0,0.210,0.492,13.5,77.6,0.192,2017,yes,113
150,1.0,San Antonio Spurs,0.817073,30.3,67,15,10.63,-0.36,10.28,110.3,...,12.4,23.0,0.197,0.477,14.1,79.1,0.182,2016,yes,108
65,6.0,New York Knicks,0.658537,30.2,53,29,4.23,-0.50,3.73,111.1,...,11.7,25.6,0.196,0.508,14.8,74.7,0.216,2013,yes,99
165,16.0,Dallas Mavericks,0.512195,30.3,40,42,-0.30,0.29,-0.02,106.7,...,12.0,20.6,0.211,0.504,12.8,76.2,0.198,2016,yes,99


In [ ]:
injury_mean = advanced_injuries['Injury Count'].mean()
advanced_injuries['injured_and_playoffs'] = advanced_injuries.apply(lambda x: 'yes' if x['playoffs'] == 'yes' and x['Injury Count'] >= injury_mean * 1.5 else 'no', axis=1)

In [ ]:
advanced_injuries
advanced_injuries['color'] = advanced_injuries.apply(lambda row: 'no playoffs' if row['playoffs'] == 'no' else ('playoffs and not injuried' if row['injured_and_playoffs'] == 'no' else 'playoffs and injuried'), axis=1)
advanced_injuries


,Rk,Team,Win Percentage,Age,PW,PL,MOV,SOS,SRS,ORtg,...,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,year,playoffs,Injury Count,injured_and_playoffs,color
116,27.0,Los Angeles Lakers,0.329268,26.8,25,57,-6.29,0.96,-5.33,104.2,...,0.199,0.513,12.4,71.0,0.192,2014,no,196,no,no playoffs
118,29.0,Milwaukee Bucks,0.182927,24.7,20,62,-8.18,-0.23,-8.41,103.0,...,0.204,0.520,13.2,71.4,0.225,2014,no,160,no,no playoffs
96,7.0,Houston Rockets,0.658537,25.4,53,29,4.56,0.50,5.06,111.0,...,0.275,0.489,12.5,74.1,0.193,2014,yes,154,yes,playoffs and injuried
79,20.0,Minnesota Timberwolves,0.378049,26.4,34,48,-2.37,0.54,-1.83,102.9,...,0.226,0.511,15.0,74.0,0.187,2013,no,136,no,no playoffs
53,24.0,New Orleans Pelicans,0.318182,25.5,24,42,-3.76,0.65,-3.11,100.9,...,0.207,0.485,13.5,73.1,0.218,2012,no,134,no,no playoffs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,17.0,San Antonio Spurs,0.450704,27.6,33,38,-1.11,0.46,-0.65,112.4,...,0.212,0.542,11.7,79.2,0.193,2020,no,24,no,no playoffs
139,20.0,Detroit Pistons,0.390244,25.9,38,44,-1.00,-0.39,-1.39,105.3,...,0.184,0.502,13.2,75.0,0.192,2015,no,23,no,no playoffs
247,8.0,Denver Nuggets,0.658537,24.9,51,31,3.95,0.24,4.19,113.0,...,0.175,0.521,12.3,78.0,0.194,2019,yes,23,no,playoffs and not injuried
296,27.0,Charlotte Hornets,0.353846,24.3,19,46,-6.75,-0.28,-7.03,106.3,...,0.188,0.546,13.1,74.4,0.159,2020,no,22,no,no playoffs


In [ ]:
import plotly.express as px

# Assuming 'advanced_injuries' is your DataFrame containing the data
fig5 = px.scatter(advanced_injuries, x='TS%', y='Injury Count', color='color',
                 title='Scatter Plot of True Shooting Percentage vs. Total Injuries',
                 labels={'TS%': 'True Shooting Percentage', 'Injury Count': 'Total Injuries'},
                 hover_data={'Team': True, 'year': True})

# Update layout
fig5.update_layout(
    xaxis_title='True Shooting Percentage',
    yaxis_title='Total Injuries'
)

# Show the plot
fig5.show()


In [ ]:
import plotly.express as px

# Create the scatter plot
fig = px.scatter(injuries_TOV, x='OTOV%', y='Injuries Count', trendline="ols", hover_data=['year'])

# Update layout
fig.update_layout(
    title='Injury Count vs. Opponent Turnover Percentage with Line of Best Fit',
    xaxis_title='Opponent Turnover Percentage (%)',
    yaxis_title='Injury Count',
    width=800,
    height=600
)

# Customize trendline
fig.update_traces(
    line=dict(color='red', width=2, dash='dashdot'),
)

# Add annotation
fig.add_annotation(
    x=13.7, y=2000,
    text="Trend Line",
    showarrow=True,
    arrowhead=1,
    ax=-50,
    ay=-50
)

# Change marker properties
fig.update_traces(marker=dict(size=10, color='blue', symbol='circle'))

# Add interactive features with year
fig.update_traces(
    hovertemplate="<b>Opponent TOV%:</b> %{x}%<br><b>Injury Count:</b> %{y}<br><b>Year:</b> %{customdata[0]}<extra></extra>"
)

# Show plot
fig.show()


In [ ]:
from sklearn.preprocessing import StandardScaler
i = advanced_injuries.sort_values("Injury Count", ascending = False)

numerical_columns = i.select_dtypes(include=['number']).columns

# Extract numerical data
numerical_data = i[numerical_columns]

scaler = StandardScaler()

i[numerical_columns] = scaler.fit_transform(numerical_data)

In [ ]:
i.corr(numeric_only=True)['Injury Count']

Rk                0.064617
Win Percentage   -0.089284
Age               0.071458
PW               -0.062130
PL                0.089248
MOV              -0.065705
SOS              -0.037537
SRS              -0.070681
ORtg             -0.159827
DRtg             -0.070882
NRtg             -0.071719
Pace             -0.149639
FTr               0.097903
3PAr             -0.085582
TS%              -0.064408
OeFG%            -0.082055
OTOV%             0.239632
ORB%             -0.013979
OFT/FGA           0.091412
DeFG%            -0.108857
DTOV%             0.084072
DRB%             -0.188847
DFT/FGA           0.071551
year             -0.213996
Injury Count      1.000000
Name: Injury Count, dtype: float64

In [ ]:
advanced_injuries = df_.merge(inj_merge, left_on = ['year', 'Team'], right_on = ['year', 'Team']).rename(columns = {'Team_y': 'Team'})
advanced_injuries.sort_values('Injury Count', ascending=False)
advanced_injuries.corr(numeric_only=True)['Injury Count']

Rk                0.064617
Win Percentage   -0.089284
Age               0.071458
PW               -0.062130
PL                0.089248
MOV              -0.065705
SOS              -0.037537
SRS              -0.070681
ORtg             -0.159827
DRtg             -0.070882
NRtg             -0.071719
Pace             -0.149639
FTr               0.097903
3PAr             -0.085582
TS%              -0.064408
OeFG%            -0.082055
OTOV%             0.239632
ORB%             -0.013979
OFT/FGA           0.091412
DeFG%            -0.108857
DTOV%             0.084072
DRB%             -0.188847
DFT/FGA           0.071551
year             -0.213996
Injury Count      1.000000
Name: Injury Count, dtype: float64

In [ ]:
best_injured = advanced_injuries.sort_values('Injury Count', ascending= False)
best_injured = best_injured[(best_injured.playoffs == 'yes') & (best_injured['Injury Count'] >= best_injured['Injury Count'].mean() * 1.5)]
best_injured

,Rk,Team,Win Percentage,Age,PW,PL,MOV,SOS,SRS,ORtg,...,OTOV%,ORB%,OFT/FGA,DeFG%,DTOV%,DRB%,DFT/FGA,year,playoffs,Injury Count
96,7.0,Houston Rockets,0.658537,25.4,53,29,4.56,0.50,5.06,111.0,...,14.6,27.4,0.275,0.489,12.5,74.1,0.193,2014,yes,154
181,2.0,San Antonio Spurs,0.743902,29.6,60,22,7.20,-0.06,7.13,111.1,...,12.6,24.0,0.210,0.492,13.5,77.6,0.192,2017,yes,113
150,1.0,San Antonio Spurs,0.817073,30.3,67,15,10.63,-0.36,10.28,110.3,...,12.4,23.0,0.197,0.477,14.1,79.1,0.182,2016,yes,108
65,6.0,New York Knicks,0.658537,30.2,53,29,4.23,-0.50,3.73,111.1,...,11.7,25.6,0.196,0.508,14.8,74.7,0.216,2013,yes,99
165,16.0,Dallas Mavericks,0.512195,30.3,40,42,-0.30,0.29,-0.02,106.7,...,12.0,20.6,0.211,0.504,12.8,76.2,0.198,2016,yes,99
212,3.0,Golden State Warriors,0.707317,28.8,56,26,5.98,-0.19,5.79,113.6,...,14.1,21.0,0.195,0.504,12.6,76.3,0.186,2018,yes,96
108,19.0,Brooklyn Nets,0.536585,30.0,38,44,-1.00,-0.58,-1.58,106.7,...,14.1,21.7,0.236,0.509,14.9,72.3,0.236,2014,yes,93
210,1.0,Houston Rockets,0.792683,29.8,61,21,8.48,-0.27,8.21,114.7,...,12.7,21.3,0.233,0.521,13.4,79.9,0.171,2018,yes,93
217,8.0,San Antonio Spurs,0.573171,29.3,49,33,2.89,0.00,2.89,107.9,...,12.2,23.7,0.189,0.507,13.2,78.6,0.170,2018,yes,91
180,1.0,Golden State Warriors,0.817073,28.2,67,15,11.63,-0.28,11.35,115.6,...,13.2,22.8,0.204,0.486,13.5,74.9,0.198,2017,yes,88


In [ ]:
best_injured.corr(numeric_only=True)['Win Percentage']

Rk               -0.920882
Win Percentage    1.000000
Age              -0.100429
PW                0.976220
PL               -0.976220
MOV               0.979004
SOS              -0.260641
SRS               0.969670
ORtg              0.841497
DRtg             -0.607251
NRtg              0.978877
Pace              0.474541
FTr              -0.173508
3PAr              0.174501
TS%               0.694336
OeFG%             0.703196
OTOV%             0.005504
ORB%              0.069185
OFT/FGA          -0.120236
DeFG%            -0.424823
DTOV%            -0.013720
DRB%              0.445261
DFT/FGA          -0.421689
year              0.348273
Injury Count      0.016263
Name: Win Percentage, dtype: float64

In [ ]:
pip install dash-vega-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.1 MB/s eta 0:00:00


In [ ]:
pip install dash


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.9 MB/s eta 0:00:00


In [ ]:
# importing modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
import warnings
import dash_vega_components as dvc
from dash import Dash, dcc, html

# use to prevent altair built-in 5000 row limit
alt.data_transformers.disable_max_rows()

# use to prevent deprecation warnings
warnings.filterwarnings('ignore')

In [ ]:
fig_1_annotation = html.P(
    html.Ul(
        [
        html.Li("Injuries generally over time are decreasing, and hit a low in the 2020 NBA Season"),
        html.Li("We can notice that Injuries spike in 2014"),
        ], style={'textAlign':'left'})
)

fig_2_annotation = html.P(
    html.Ul(
        [
        html.Li("There was a much larger discrepancy between injuries in earlier years as compared to more recent injuries, suggesting that there may be better resting protocol and better player load management in place to protect players"),
        html.Li("There was no correlation between teams and the injuries that they faced/didn’t face over the 10-year dataset"),
        html.Li("The top 3 teams with the highest injury count throughout the 10-year dataframe (Lakers, Bucks, Rockets) all occurred in 2014")
        ], style={'textAlign':'left'})
)

fig_3_annotation = html.P(
    html.Ul(
        [
        html.Li("Pace and Injury Count did not have a correlation after creating the 3-D graph this could be due to the fact that team depth allows for less injuries or burnout"),
        html.Li("Year and Injury Count: Early years Injuries impact playoff admission, however in recent years injuries have not affected playoff admission - due to sports medicine practices and rehabilitation efforts"),
        html.Li("In recent years teams who are impacted by injury have also made the playoffs."),
        ], style={'textAlign':'left'})
)

fig_4_annotation = html.P(
    html.Ul(
        [
        html.Li("NBA Season is October-May. We can see a constant increase in injuries until February."),
        html.Li("February's all star weekend causes less games meaning less players getting injured. Decline after march as players sit in less important games and teams get elimiated."),
        html.Li("We performed a Case study of the 2015-2020 Golden State Warriors and saw they had the greatest amount of injuries but also the greatest success through the strech in recent NBA history (4 finals apperances in these 5 years)")

        ], style={'textAlign':'left'})
)

fig_5_annotation = html.P(
    html.Ul(
        [
        html.Li("Teams that make the playoffs generally have a high TS% "),
        html.Li("Teams with a high injury count (1.5 standard deviations above the mean) still make the playoffs if they have a high TS%"),
        ], style={'textAlign':'left'})
)
conc = html.P(
    html.Ul(
        [
        html.Li("Injuries have been shown to be fairly random overtime, and that outside of basic load management systems, there’s little that teams can do to prevent injuries from impacting their rosters"),
        html.Li("Teams can reduce the effect of injuries by focusing more on outside shooting and overall being more offensively-minded"),
        html.Li("Teams will have better chances of making the playoffs if they build their roster around shooting 3 pointers and having a more modern style of play"),
        ], style={'textAlign':'left'})
)

In [ ]:
app = Dash()
app.layout = html.Div([
    html.H1("Injuries through Evolution"),
    html.H2("Aiman Lakhani (al54554), Illiyan Bardai (isb354) & Iliyan Tharani (irt258)"),
    html.H3("April 16, 2024. Group 10"),
    html.Hr(),
    html.H1("Visual #1"),
    dvc.Vega(
                id="viz-1",
                opt={"renderer": "svg", "actions": False},
                spec=chart.to_dict()),
                fig_1_annotation,
    html.Hr(),
    html.H1("Visual #2"),
    dvc.Vega(
                id="viz-2",
                opt={"renderer": "svg", "actions": False},
                spec=combined_chart_2.to_dict()),
                fig_2_annotation,
    html.Hr(),
    html.H1("Visual #3"),
    dcc.Graph(figure = fig_3),
    fig_3_annotation,
    html.Hr(),



    html.H1("Visual #4"),
            dvc.Vega(
                id="viz-4",
                opt={"renderer": "svg", "actions": False},
                spec=combined_charts_with_annotation4.to_dict()),
                fig_4_annotation,
    html.Hr(),

    html.H1("Visual #5"),
    dcc.Graph(figure = fig5),
    fig_5_annotation,
    html.Hr(),
    html.H1("Key Takeaways"),
    conc,
    html.Hr(),],
                      style={'textAlign': 'center'})

app.run_server(debug = True, use_reloader = False, jupyter_mode='external')

Dash app running on:


<IPython.core.display.Javascript object>